# Tutorial 1 Dataframes en Pyspark
## Érick Alejandro Muñoz Alvarado
## 2019

## Imports necesarios
Nota: Varias líneas pueden tirar errorres o salidas diferentes a las que esperamos si no se está usando Java 8 ya que es a versión default de spark

In [1]:
from pyspark.sql import SparkSession

## Desarrollo del código

Para el uso de dataframes es necesario crear una sesión de spark como se explica a continuación:

    * Uso del "SparkSession.builder" como contructor de la sesión.
    * El ".master" indica la conexión a el servidor, en este caso local, que será utilizado como cluster de procesamiento.
    * Tercero, el ".appName" define el nombre de la sesión, en nuestro caso será "dfTest".
    * Y finalmente el ".getOrCreate" busca la sessión de spark y de no existir, crea una nueva basada en las características brindadas y lo retorna.

In [2]:
# Creación de la conexión
spark = SparkSession.builder \
        .master("local") \
        .appName("dfTest") \
        .getOrCreate()

# Verificcamos que la creación haya sido un éxito
print(10*"-" + "Sesión de spark creada satifactoriamente" + 10*"-" + '\n')
print(spark)
print('\n' + 60*"-")

----------Sesión de spark creada satifactoriamente----------


------------------------------------------------------------


Seguidamente crearemos un pequeño dataframe basado en un CSV sobre los videos que fueron trending en Youtube México.

In [3]:
# Ubicación del archivo que vamos a leer
mx = 'MXvideos.csv'
ca = 'CAvideos.csv'

# Lectura del archivo en forma de dataframe 
# Con header=True saltamos la primer fila puesto que corresponde al header del archivo
# Con inferSchema = True, infermos los tipos de datos de cada columna
mx_df = spark.read.csv(mx, header=True, inferSchema = True)
ca_df = spark.read.csv(ca, header=True, inferSchema = True) 

# Verificamos que se haya cargado correctamente
print(type(mx_df))
print(type(ca_df))

<class 'pyspark.sql.dataframe.DataFrame'>
<class 'pyspark.sql.dataframe.DataFrame'>


Spark nos ofrece una gran facilidad cuando se trata de opeerar sobre dataframes, posee una gran cantidad de operaciones y la gran mayoría, está altemente optimizada.

In [4]:
mx_df.show()
mx_df.printSchema()

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|SbOwzAl9ZfQ|     17.14.11|Capítulo 12 | Mas...|     MasterChef 2017|         24|2017-11-13T06:06:...|"MasterChef Junio...| 310130| 4182|     361|         1836|https://i.ytimg.c...|            FALSE|           FALSE| 

Dado que ya conocemos lo anterior podemos empear a trabajar los datos.

1.Para observar los primeros N elementos 

In [5]:
N = 5
print('Primeros ' + str(N) +' elementos de mx_df')
mx_df.head(N)

Primeros 5 elementos de mx_df


[Row(video_id='SbOwzAl9ZfQ', trending_date='17.14.11', title='Capítulo 12 | MasterChef 2017', channel_title='MasterChef 2017', category_id='24', publish_time='2017-11-13T06:06:22.000Z', tags='"MasterChef Junior 2017|""TV Azteca""|""recetas""|""cocina""|""Anette Michel""|""Betty Vázquez""|""Benito Molina""|""Adrián Herrera""|""master chef mexico""|""masterchef""|""master chef""|""master chef mexico 2017""|""masterchef mexico jueces""|""la cocina más famosa de méxico""|""reality de cocina más famoso de méxico""|""capítulo 12""|""episodio 12""|""capitulo masterchef""|""capitulo masterchef mexico 2017""|""programa masterchef de hoy"""', views='310130', likes='4182', dislikes='361', comment_count='1836', thumbnail_link='https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg', comments_disabled='FALSE', ratings_disabled='FALSE', video_error_or_removed='FALSE', description='Disfruta la presencia del Chef Torreblanca en MasterChef, el duelo de pasteles, las mesas regionales de los chefs Benito, Betty 

2.Para contar la cantidad de filas que tiene el df

In [6]:
num_filas = mx_df.count()
print('Cantidad de filas que tiene el df es ' + str(num_filas))

Cantidad de filas que tiene el df es 43819


3.Columnas en el df


In [7]:
cols = mx_df.columns
print(cols)

['video_id', 'trending_date', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 'views', 'likes', 'dislikes', 'comment_count', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'video_error_or_removed', 'description']


4.Obtiene el resumen estadístico de las columnas numéricas del dataframe.


In [8]:
mx_df.describe().show()

+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+--------------------+----------------+----------------------+--------------------+
|summary|            video_id|  trending_date|               title|       channel_title|         category_id|        publish_time|                tags|             views|               likes|          dislikes|     comment_count|      thumbnail_link|   comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+------------------+------------------+--------------------+--------------------+----------------+----------------------+--------------------+
|

5.Obtiene el resumen estadístico de la columna específica del dataframe.

In [9]:
mx_df.describe('views').show()

+-------+------------------+
|summary|             views|
+-------+------------------+
|  count|             40478|
|   mean| 342381.9681590072|
| stddev|1714690.6809946736|
|    min|          Chiclayo|
|    max|            999860|
+-------+------------------+



6.Seleccionar una columna de un dataframe.

In [10]:
mx_df.select('description').show(5)

+--------------------+
|         description|
+--------------------+
|Disfruta la prese...|
|ALEXA EX-INTEGRAN...|
|La canción del pr...|
|El video es de un...|
|MI HERMANO NARRA ...|
+--------------------+
only showing top 5 rows



7.Cantidad de filas distintas en una columna.

In [11]:
mx_df.select('publish_time').distinct().count()

33259

8.Para eliminar filas con valores nulos.

9.Para sustituit los valores nulos por una constante.

In [12]:
mx_df.dropna().count()

36246

9.Para sustituir los valores nulos por una constante, en este caso 0.

In [13]:
mx_df.fillna(0).count()

43819

10.Filtrar las filas que cumplen cierta característica, en este caso los que tienen más de un millón de vistas.

In [14]:
mx_df.filter(mx_df.views > 1000000).count()

2544

11.Agrupar y operar, en este caso agrupamos por categoría y sacamos al media de vistas de cada una.

In [15]:
mx_df.dropna().groupby('category_id').agg({'views': 'mean'}).show()

+--------------------+------------------+
|         category_id|        avg(views)|
+--------------------+------------------+
|                  15| 548242.8266666667|
|                  29|248352.35211267605|
|         reptilianos|              null|
| duelos lejos de ...|              null|
|                  22|        166486.484|
|                  28| 397307.9166666667|
|                  43|           21812.0|
|                  27| 86884.64826175869|
|                  17|334962.48679553496|
|                  26|192539.60165975103|
|             Jessica|              null|
|                  19|120978.20353982301|
|                  23| 480729.1413515189|
|                  25|164414.30886956523|
|        Don Barrabas|              null|
|                  24| 320069.6200097768|
|                   1| 387496.7376910017|
|                  20| 454563.6822742475|
|                  10| 1280763.882843895|
|                   2|197129.26829268291|
+--------------------+------------

12.Ordenar basado en una columna.

In [16]:
mx_df.orderBy(mx_df.views, ascending=False).show(5)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags| views|likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+------+-----+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|iiOzcMQiDZs|     17.27.11|Valencia vs Barce...|           GOLAZO TV|         17|2017-11-26T21:50:...|"Valencia|""vs""|...|999860| 5586|     481|         1693|https://i.ytimg.c...|            FALSE|           FALSE|     

13.Hacer el drop de una columna.

In [17]:
mx_df.drop("tags").columns

['video_id',
 'trending_date',
 'title',
 'channel_title',
 'category_id',
 'publish_time',
 'views',
 'likes',
 'dislikes',
 'comment_count',
 'thumbnail_link',
 'comments_disabled',
 'ratings_disabled',
 'video_error_or_removed',
 'description']

14.Eliminar videos que parecen tanto en la lista de México como la de Canadá.

In [18]:
diff_df = mx_df.select('video_id').subtract(ca_df.select('video_id'))
diff_df.distinct().count()

32473

15.Join.

In [19]:
mx_df.join(ca_df, mx_df.video_id == ca_df.video_id).show(10)

+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+--------------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|video_error_or_removed|         description|   video_id|trending_date|               title|       channel_title|category_id|        publish_time|                tags|  views| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|v

Join derecho, izquierdo y externo.

In [20]:
mx_df.join(ca_df, mx_df.video_id == ca_df.video_id, how='right').show(10)# Igual a 'full_right'
mx_df.join(ca_df, mx_df.video_id == ca_df.video_id, how='left').show(10)# Igual a 'full_left'
mx_df.join(ca_df, mx_df.video_id == ca_df.video_id, how='full').show(10)# Igual a 'full_outer'

led|ratings_disabled|video_error_or_removed|         description|
+-----------+-------------+--------------------+----------------+-----------+--------------------+--------------------+-------+------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+-----------+-------------+--------------------+----------------+-----------+--------------------+--------------------+--------+-------+--------+-------------+--------------------+-----------------+----------------+----------------------+--------------------+
|       null|         null|                null|            null|       null|                null|                null|   null|  null|    null|         null|                null|             null|            null|                  null|                null|2Vv-BfVoq4g|     17.14.11|Ed Sheeran - Perf...|      Ed Sheeran|         10|2017-11-09T11:04:...|"edsheeran"|"ed s...|33523622|1634130|   21082|        85067|htt

## Ejecución de queries SQL

En este caso debemos operar diectamente sobre el contexto o la sesión de spark que estemos utilizando en nuestro caso es una sesión y se ejecuta de la siguiente manera:

1.Agregamos el dataframe como una vista temporal.

In [21]:
mx_df.createOrReplaceTempView('mx_table')

2. Ejecutamos el querie.

In [22]:
querie = 'SELECT video_id FROM mx_table'
spark.sql(querie).show(5)

+-----------+
|   video_id|
+-----------+
|SbOwzAl9ZfQ|
|klOV6Xh-DnI|
|6L2ZF7Qzsbk|
|hcY52MFWMDM|
|_OXDcGPVAa4|
+-----------+
only showing top 5 rows



# Refefencias 
1. https://www.kaggle.com/datasnaek/youtube-new
2. https://spark.apache.org/docs/latest/api/python/index.html